# 文本生成(中文歌詞)
- Author: Lynn
- Updated: 2021/2/7

- 參考資料：
    - https://leemeng.tw/how-to-generate-interesting-text-with-tensorflow2-and-tensorflow-js.html
    - https://www.tensorflow.org/tutorials/text/text_generation

In [10]:
from google.colab import drive 
drive.mount('/content/gdrive')
data_filepath = '/content/gdrive/MyDrive/AI_&_EdgeComputing_Program/NLP/shared_folder/dataset/top300.txt'

# 將整個檔案讀入後存放為一個字串
with open(data_filepath) as f:
    text = f.read()

print(type(text)) 
#<class 'str'>
print('text[:100]: ',text[:100])
#text[:100]:  Your Name Engraved Herein 電影 刻在你心底的名字 主題曲 作曲：許媛婷、佳旺、陳文華 Oublié-le 好幾次我告訴我自己 越想努力趕上光的影 越無法抽離  而已 Je t

text = text.replace('更多更詳盡歌詞','')
text = text.replace('魔鏡歌詞網','')
text = text.replace('作曲：','')

print(f'共有{len(text)}個字')
#共有163065個字

print('清理前的字串：')
print(text[:100])

import jieba
from string import punctuation

my_stop_words = ['：','。','。','、','※','【','】']

clean_sentences = []

# 每首歌為一個句子，分詞後處理，再接起來
for sent in text.split('\n'): #先用換行符號分割 一首歌當做一個句子
    clean_tokens = []
    for token in jieba.lcut(sent): #(使用結巴分詞)
        # 移除英文、標點、空白 #技巧!!!處理中英混雜  #將element encode然後判斷是不是英文 為何要encode? #非punctuatuation半形標點 #停用詞
        if token.strip() != '' and not token.encode().isalpha() and not token in punctuation and not token in my_stop_words:
            clean_tokens.append(token)
    if len(clean_tokens)>0:
        clean_sent = ''.join(clean_tokens)
        clean_sentences.append(clean_sent)

# 不同首歌之間以句點串接
new_text = '。'.join(clean_sentences)

print('清理後的字串：')
print(new_text[:100])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
<class 'str'>
text[:100]:  Your Name Engraved Herein 電影 刻在你心底的名字 主題曲 作曲：許媛婷、佳旺、陳文華 Oublié-le 好幾次我告訴我自己 越想努力趕上光的影 越無法抽離  而已 Je t
共有163065個字
清理前的字串：
Your Name Engraved Herein 電影 刻在你心底的名字 主題曲 許媛婷、佳旺、陳文華 Oublié-le 好幾次我告訴我自己 越想努力趕上光的影 越無法抽離  而已 Je t'ai
清理後的字串：
電影刻在你心底的名字主題曲許媛婷佳旺陳文華é好幾次我告訴我自己越想努力趕上光的影越無法抽離而已刻骨銘心只有我自己好不容易交出真心的勇氣你沉默的回應是善意刻在我心底的名字忘記了時間這回事於是謊言說了一次


In [11]:
from keras.preprocessing.text import Tokenizer

# 初始化 Tokenizer，以字為單位
tokenizer = Tokenizer(char_level=True)

# 建辭典(字與索引對應)
tokenizer.fit_on_texts(new_text)

# *** vocab_size 為何要加1? 
# https://keras.io/api/layers/core_layers/embedding/#embedding
# https://github.com/keras-team/keras/issues/9637
"""詳見(keras.io/api/layers/core_layers/embedding/#embedding)"""


#我們要利用word embedding層，其sizeg是詞典長度
#vocab_size = len(tokenizer.word_index)+1， 加1原因為加一回去0的保留字 (老師沒說原因)    
#記得 idx最小是1 因為第0個是保留字 #可用len取代max
vocab_size = max([idx for word,idx in tokenizer.word_index.items()])+1
print(f'詞典中共有{vocab_size}個詞')         #word_index取出dict 利用item取出key , value


# 將文字轉成數字

# (作法一：text_ids 為 1D)
# ([241, 227, 180, 7, 3, 15, 228, 1, 500, 422, 361, ...])
text_ids = tokenizer.texts_to_sequences([new_text])[0] #將句子中的詞轉成詞編號， [0]表示去掉2D array最外層的[] 變成1D
print(text_ids)

# (作法二：text_ids 為 2D)
# ([[241], [227], [180], [7], [3], [15], [228], [1], [500], [422], ..])
#text_ids = tokenizer.texts_to_sequences(new_text)
#print(text_ids)

print(f'語料庫中共有{len(text_ids)}個詞')

# 文字與數字互轉
idx = tokenizer.word_index['愛']
w = tokenizer.index_word[idx]
print(idx)
print(w)

詞典中共有3223個詞
[241, 227, 180, 7, 3, 15, 228, 1, 500, 422, 361, 457, 263, 239, 2294, 1584, 1219, 2066, 501, 663, 508, 1854, 27, 235, 169, 2, 394, 440, 2, 48, 128, 181, 9, 576, 147, 728, 55, 107, 1, 227, 181, 36, 158, 960, 122, 132, 165, 180, 925, 1220, 15, 43, 8, 2, 48, 128, 27, 4, 300, 485, 706, 87, 81, 15, 1, 291, 150, 3, 325, 252, 1, 60, 395, 6, 1004, 112, 180, 7, 2, 15, 228, 1, 500, 422, 133, 109, 10, 41, 67, 25, 60, 183, 195, 6, 519, 307, 23, 10, 5, 169, 21, 5, 787, 97, 151, 1221, 1005, 264, 74, 113, 54, 2686, 153, 39, 407, 400, 526, 20, 6, 1088, 1222, 50, 139, 8, 57, 169, 2, 14, 37, 11, 5, 169, 180, 7, 2, 15, 228, 1, 500, 422, 3, 396, 7, 502, 744, 1, 926, 1153, 16, 4, 6, 25, 75, 2, 93, 38, 17, 5, 787, 97, 2, 260, 7, 1855, 539, 1, 362, 551, 577, 12, 166, 237, 19, 846, 1, 88, 2295, 3, 58, 71, 1585, 788, 58, 6, 2, 43, 28, 208, 2687, 373, 211, 3, 623, 623, 13, 140, 68, 118, 9, 52, 3, 7, 27, 4, 300, 485, 122, 56, 310, 159, 1, 1856, 664, 60, 200, 240, 2, 407, 2688, 24, 17, 34, 180, 7, 2, 

### 訓練階段

In [12]:
import tensorflow as tf
import numpy as np

# 序列長度(以此長度預測下個字)
SEQ_LENGTH = 10 #以前10個詞預測第11個



# 將list轉成slices 將串列轉切片
characters = tf.data.Dataset.from_tensor_slices(text_ids)

# 將字串拆成多個長度為 SEQ_LENGTH 的序列，並將最後長度不滿 SEQ_LENGTH 的序列捨去
sequences = characters.batch(SEQ_LENGTH+1,drop_remainder=True)

# 全文所包含的序列數量  #SEQ_LENGTH?????
steps_per_epoch = len(text_ids)//SEQ_LENGTH

# 將單一序列拆成兩個部分(錯位?)，分別為輸入與輸出內容
def build_seq_pairs(chunk): #chunk是長度為11的串列 看影片澄清觀念?????
    input_text = chunk[:-1] #前10個字 (不包含最後一個, 0~倒數第二個)
    target_text = chunk[1:] #後10個字 (錯位一個字)
    return input_text, target_text

# 將每個序列拆成兩個部分，作為輸入與輸出序列 #map 用法約等同pd.apply() #build_seq_pairs???????
pairs = sequences.map(build_seq_pairs)

# 將得到的所有資料隨機打亂順序shuffle
rand_pairs = pairs.shuffle(steps_per_epoch)


BATCH_SIZE = 16 #假設

# 取出 BATCH_SIZE(16)筆數據，作為模型一次訓練步驟的所使用的資料
ds = rand_pairs.batch(BATCH_SIZE,drop_remainder=True)
#ds 通常為dataset縮寫

In [13]:
# 觀察BATCH中的資料
for b_inp, b_tar in ds.take(1):
    print("第一個起始句子的索引序列：")
    first_i = b_inp.numpy()[0]
    print(first_i, "\n")
    print("第一個目標句子的索引序列：")
    first_t = b_tar.numpy()[0]
    print(first_t, "\n")
    print("-" * 20, "\n")
    
    d = tokenizer.index_word
    print('-'*10)
    print(d)
    
    print("第一個起始句子的文本序列：")
    print([d[i] for i in first_i])
    print()
    print("第一個目標句子的文本序列：")
    print([d[i] for i in first_t])

第一個起始句子的索引序列：
[  24  212  212  196  193  280  147 1053   87   10] 

第一個目標句子的索引序列：
[ 212  212  196  193  280  147 1053   87   10  465] 

-------------------- 

----------
{1: '的', 2: '我', 3: '你', 4: '不', 5: '一', 6: '是', 7: '在', 8: '有', 9: '想', 10: '了', 11: '愛', 12: '著', 13: '人', 14: '會', 15: '心', 16: '要', 17: '過', 18: '來', 19: '天', 20: '都', 21: '就', 22: '-', 23: '說', 24: '到', 25: '這', 26: '個', 27: '好', 28: '能', 29: '沒', 30: '們', 31: '為', 32: '那', 33: '還', 34: '去', 35: '多', 36: '無', 37: '再', 38: '麼', 39: '得', 40: '也', 41: '時', 42: '看', 43: '只', 44: '讓', 45: '最', 46: '走', 47: '裡', 48: '自', 49: '情', 50: '如', 51: '生', 52: '起', 53: '眼', 54: '對', 55: '上', 56: '開', 57: '下', 58: '可', 59: '後', 60: '回', 61: '。', 62: '別', 63: '像', 64: '手', 65: '妳', 66: '見', 67: '間', 68: '卻', 69: '感', 70: '中', 71: '以', 72: '風', 73: '點', 74: '世', 75: '樣', 76: '誰', 77: '笑', 78: '身', 79: '大', 80: '當', 81: '真', 82: '他', 83: '和', 84: '太', 85: '成', 86: '道', 87: '出', 88: '地', 89: '給', 90: '年', 91: '放', 92: '夜', 93: '怎', 9

In [14]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense

# 設定超參數
EMBEDDING_DIM = 512
RNN_UNITS = 1024

# 建立模型
model = Sequential()

# 詞嵌入層，將每個索引數字對應到一個低維空間向量
model.add(Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, batch_input_shape=[BATCH_SIZE, None]))

# LSTM 層，將序列數據依序讀入並做處理 #這邊只有單向 因為我們用前面預測後面的詞

model.add(LSTM(units=RNN_UNITS, return_sequences=True, stateful=True))
                            ##stateful 指承接上一個hidden state (有疑義?)

# 全連接層，計算每個中文字出現的可能性
model.add(Dense(vocab_size,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (16, None, 512)           1650176   
_________________________________________________________________
lstm (LSTM)                  (16, None, 1024)          6295552   
_________________________________________________________________
dense (Dense)                (16, None, 3223)          3303575   
Total params: 11,249,303
Trainable params: 11,249,303
Non-trainable params: 0
_________________________________________________________________


In [15]:
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# 編譯模型
LEARNING_RATE = 0.001

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [16]:
# 訓練模型
EPOCHS = 10
history = model.fit(ds,epochs=EPOCHS)

Epoch 1/10
651/651 [==============================] - 41s 29ms/step - loss: 6.4584 - accuracy: 0.0554
Epoch 2/10
651/651 [==============================] - 19s 29ms/step - loss: 5.1058 - accuracy: 0.1699
Epoch 3/10
651/651 [==============================] - 19s 29ms/step - loss: 4.1274 - accuracy: 0.2644
Epoch 4/10
651/651 [==============================] - 19s 29ms/step - loss: 3.2487 - accuracy: 0.3814
Epoch 5/10
651/651 [==============================] - 19s 29ms/step - loss: 2.5633 - accuracy: 0.4983
Epoch 6/10
651/651 [==============================] - 19s 29ms/step - loss: 2.0650 - accuracy: 0.5957
Epoch 7/10
651/651 [==============================] - 19s 29ms/step - loss: 1.7069 - accuracy: 0.6672
Epoch 8/10
651/651 [==============================] - 19s 29ms/step - loss: 1.4698 - accuracy: 0.7176
Epoch 9/10
651/651 [==============================] - 19s 29ms/step - loss: 1.3100 - accuracy: 0.7516
Epoch 10/10
651/651 [==============================] - 19s 29ms/step - loss: 1.200

In [21]:
# 儲存模型
model.save("/content/drive/MyDrive/AI_&_EdgeComputing_Program/NLP/shared_folder/dataset/lm_model_2.h5")

### 生成(預測)階段

In [18]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense

# 設定超參數
EMBEDDING_DIM = 512
RNN_UNITS = 1024
BATCH_SIZE = 1

# 建立模型
infer_model = Sequential()
infer_model.add(Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, batch_input_shape=[BATCH_SIZE, None]))
infer_model.add(LSTM(units=RNN_UNITS,return_sequences=True,stateful=True))
infer_model.add(Dense(vocab_size))

infer_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 512)            1650176   
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           6295552   
_________________________________________________________________
dense_1 (Dense)              (1, None, 3223)           3303575   
Total params: 11,249,303
Trainable params: 11,249,303
Non-trainable params: 0
_________________________________________________________________


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# 載入之前訓練好的模型的weights
infer_model.load_weights("/content/drive/MyDrive/AI_&_EdgeComputing_Program/NLP/shared_folder/dataset/lm_model_2.h5")


In [25]:
def generate_token(seed_idx):
    # 起始字的索引
    input = tf.expand_dims([seed_idx], axis=0)
    
    # 預測/生成(詞典中3223個字的機率分佈) 
    predictions = infer_model(input)
    predictions = tf.squeeze(predictions, 0)
    
    #詳見 LeeMing的blog:手動調整生成溫度
    # 溫度越高，使懸殊的機率分布，分佈會變得越平滑，罕見字被選到的機會上升，生成結果越隨機
    temperature = 0.8   #0-1的浮點數 為何除0.x可以變平滑??? #詳見 LeeMing的blog
    predictions /= temperature

    # 從3223個分類中做抽樣，取得生成的中文字
    sampled_index = tf.random.categorical(predictions, num_samples=1)
    print(sampled_index)
    
    return int(sampled_index)

In [26]:
import tensorflow as tf
import numpy as np

# 測試
init_seed_idx = 11 #愛的編號
token = tokenizer.index_word[init_seed_idx] #index_word讓編號變成詞  word_index詞變編號
print(f'起始字：{token}')

tokens = []
tokens.append(token)

seed_idx = init_seed_idx

# 產生10個字的句子
for i in range(10):
    sampled_idx = generate_token(seed_idx)    
    token = tokenizer.index_word[sampled_idx]
    tokens.append(token)
    seed_idx = sampled_idx #用來當作下一輪的輸入 (RNN)

print(''.join(tokens))

起始字：愛
tf.Tensor([[4]], shape=(1, 1), dtype=int64)
tf.Tensor([[87]], shape=(1, 1), dtype=int64)
tf.Tensor([[59]], shape=(1, 1), dtype=int64)
tf.Tensor([[6]], shape=(1, 1), dtype=int64)
tf.Tensor([[747]], shape=(1, 1), dtype=int64)
tf.Tensor([[1687]], shape=(1, 1), dtype=int64)
tf.Tensor([[1687]], shape=(1, 1), dtype=int64)
tf.Tensor([[204]], shape=(1, 1), dtype=int64)
tf.Tensor([[77]], shape=(1, 1), dtype=int64)
tf.Tensor([[330]], shape=(1, 1), dtype=int64)
愛不出後是洋娃娃微笑久
